In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
from pyspark import SparkConf,SparkContext

In [8]:
conf=SparkConf().setAppName("Rdd-Join").setMaster("local[4]")

In [9]:
sc=SparkContext(conf=conf)

### Order Table

In [15]:
order_items_rdd=sc.textFile("C:\\Users\\ferha\\Desktop\\udemy-apache-spark-master\\data\\retail_db\\order_items.csv")
order_items_rdd.take(2)

['orderItemName,orderItemOrderId,orderItemProductId,orderItemQuantity,orderItemSubTotal,orderItemProductPrice',
 '1,1,957,1,299.98,299.98']

In [19]:
order_items_rdd=order_items_rdd.filter(lambda x:"orderItemName" not in x)
order_items_rdd.take(2)

['1,1,957,1,299.98,299.98', '2,2,1073,1,199.99,199.99']

### Products Table

In [21]:
#products table 
products_rdd=sc.textFile("C:\\Users\\ferha\\Desktop\\udemy-apache-spark-master\\data\\retail_db\\products.csv")
products_rdd.take(2)

['productId,productCategoryId,productName,productDescription,productPrice,productImage',
 '1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy']

In [23]:
products_rdd=products_rdd.filter(lambda x: "productId" not in x)
products_rdd.take(2)

['1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy',
 "2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat"]

### Pair Rdd

In [26]:
def make_orderItems_pairRdd(line):
    orderItemName = line.split(",")[0]
    orderItemOrderId= line.split(",")[1]
    orderItemProductId= line.split(",")[2]
    orderItemQuantity= line.split(",")[3]
    orderItemSubTotal= line.split(",")[4]
    orderItemProductPrice= line.split(",")[5]
    
    return (orderItemProductId,(orderItemName,orderItemOrderId,orderItemQuantity,orderItemSubTotal,orderItemProductPrice))

In [27]:
order_items_pairRdd=order_items_rdd.map(make_orderItems_pairRdd)

In [28]:
order_items_pairRdd.take(3)

[('957', ('1', '1', '1', '299.98', '299.98')),
 ('1073', ('2', '2', '1', '199.99', '199.99')),
 ('502', ('3', '2', '5', '250.0', '50.0'))]

In [30]:
def make_product_pairRdd(line):
    productId = line.split(",")[0]
    productCategoryId = line.split(",")[1]
    productName = line.split(",")[2]
    productDescription = line.split(",")[3]
    productPrice = line.split(",")[4]
    productImage = line.split(",")[5]
    
    return (productId,(productCategoryId,productName,productDescription,productPrice,productImage))

In [31]:
products_pairRdd = products_rdd.map(make_product_pairRdd)
products_pairRdd.take(3)

[('1',
  ('2',
   'Quest Q64 10 FT. x 10 FT. Slant Leg Instant U',
   '',
   '59.98',
   'http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy')),
 ('2',
  ('2',
   "Under Armour Men's Highlight MC Football Clea",
   '',
   '129.99',
   'http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat')),
 ('3',
  ('2',
   "Under Armour Men's Renegade D Mid Football Cl",
   '',
   '89.99',
   'http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat'))]

### Join

In [32]:
order_items_products_pairRdd=order_items_pairRdd.join(products_pairRdd)

In [33]:
order_items_products_pairRdd.take(5)

[('957',
  (('1', '1', '1', '299.98', '299.98'),
   ('43',
    "Diamondback Women's Serene Classic Comfort Bi",
    '',
    '299.98',
    'http://images.acmesports.sports/Diamondback+Women%27s+Serene+Classic+Comfort+Bike+2014'))),
 ('957',
  (('9', '5', '1', '299.98', '299.98'),
   ('43',
    "Diamondback Women's Serene Classic Comfort Bi",
    '',
    '299.98',
    'http://images.acmesports.sports/Diamondback+Women%27s+Serene+Classic+Comfort+Bike+2014'))),
 ('957',
  (('12', '5', '1', '299.98', '299.98'),
   ('43',
    "Diamondback Women's Serene Classic Comfort Bi",
    '',
    '299.98',
    'http://images.acmesports.sports/Diamondback+Women%27s+Serene+Classic+Comfort+Bike+2014'))),
 ('957',
  (('15', '7', '1', '299.98', '299.98'),
   ('43',
    "Diamondback Women's Serene Classic Comfort Bi",
    '',
    '299.98',
    'http://images.acmesports.sports/Diamondback+Women%27s+Serene+Classic+Comfort+Bike+2014'))),
 ('957',
  (('34', '12', '1', '299.98', '299.98'),
   ('43',
    "Diamondb